<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [104]:
from getpass import getpass
HF_TOKEN = getpass("Enter your Hugging Face token: ")

Enter your Hugging Face token: ··········


In [105]:
from huggingface_hub import login
login(token=HF_TOKEN, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [106]:
import os, sys, shutil

os.chdir("/content")
current_dir = os.getcwd()
print(f"The current directory is {current_dir}")

if os.path.isdir('ML_GitHub'):
    shutil.rmtree('ML_GitHub')

!git init
!git branch -m main
!git clone https://github.com/claudelepere/ML_GitHub.git
print(f"list /content: {os.listdir(current_dir)}")
print(f"list /content/ML_GitHub: {os.listdir(current_dir + '/ML_GitHub')}")
!ls -la /content/ML_GitHub/datasetHF
os.chdir(current_dir + '/ML_GitHub')

#!pip install fsspec==2024.10.0
!pip install -q transformers datasets
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk('datasetHF')
print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")

The current directory is /content
Reinitialized existing Git repository in /content/.git/
Cloning into 'ML_GitHub'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 62 (delta 27), reused 50 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 9.97 MiB | 13.21 MiB/s, done.
Resolving deltas: 100% (27/27), done.
list /content: ['.config', 'ML_GitHub', '.git', 'sample_data']
list /content/ML_GitHub: ['README.md', 'Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb', 'datasetHF', '.git']
total 24
drwxr-xr-x 5 root root 4096 Nov  6 12:43 .
drwxr-xr-x 4 root root 4096 Nov  6 12:43 ..
-rw-r--r-- 1 root root   43 Nov  6 12:43 dataset_dict.json
drwxr-xr-x 2 root root 4096 Nov  6 12:43 test
drwxr-xr-x 2 root root 4096 Nov  6 12:43 train
drwxr-xr-x 2 root root 4096 Nov  6 12:43 validation
dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (12

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's check the first example of the training split:

In [118]:
example = dataset['validation'][0]
example

{'id': 141545,
 'text': 'Actiris - Scrum Master / Gestionnaire de Projets Scrum Master, Chef de Projets, .NET Actiris Le service développement d\'Actiris est composé d\'une trentaine de personnes qui travaillent essentiellement sur des projets nouveaux (dossier unique, plateforme pour la gestion des relations entre Actiris et les partenaires, plateforme pour la gestion des relations entre Actiris et les employeurs, mise en place d\'un nouveau CRM, etc.). Votre rôle En tant que Scrum Master/gestionnaire de projets, vous initiez et coordonnez, dans un environnement Agile, des projets dans le cadre du développement de nos nouvelles plateformes core business et vous veillez à leur bonne réalisation dans les délais impartis. Vos responsabilités Vous collectez les informations nécessaires au bon déroulement du projet et êtes capable de discuter d\'un backlog avec le métier ; Vous déterminez les étapes clés, planning et rétro-planning Highlevel sans forcément de requête auprès de l\'équipe de

The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [108]:
labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
print(len(labels), list(enumerate(labels)))
id2label = {idx:label for idx, label in enumerate(labels)}
print(id2label)
label2id = {label:idx for idx, label in enumerate(labels)}
labels.sort()
print(len(labels), labels)

42 [(0, '394'), (1, '142'), (2, '146'), (3, '147'), (4, '148'), (5, '149'), (6, '150'), (7, '151'), (8, '408'), (9, '409'), (10, '153'), (11, '154'), (12, '155'), (13, '156'), (14, '157'), (15, '158'), (16, '160'), (17, '152'), (18, '162'), (19, '165'), (20, '167'), (21, '168'), (22, '169'), (23, '170'), (24, '171'), (25, '685'), (26, '174'), (27, '686'), (28, '176'), (29, '689'), (30, '173'), (31, '356'), (32, '360'), (33, '361'), (34, '362'), (35, '364'), (36, '760'), (37, '756'), (38, '758'), (39, '375'), (40, '376'), (41, '761')]
{0: '394', 1: '142', 2: '146', 3: '147', 4: '148', 5: '149', 6: '150', 7: '151', 8: '408', 9: '409', 10: '153', 11: '154', 12: '155', 13: '156', 14: '157', 15: '158', 16: '160', 17: '152', 18: '162', 19: '165', 20: '167', 21: '168', 22: '169', 23: '170', 24: '171', 25: '685', 26: '174', 27: '686', 28: '176', 29: '689', 30: '173', 31: '356', 32: '360', 33: '361', 34: '362', 35: '364', 36: '760', 37: '756', 38: '758', 39: '375', 40: '376', 41: '761'}
42 ['14

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [109]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# examples and not example because batched=True => examples is a batch
def preprocess_data(examples, indices):
  # take a batch of texts
  text = examples["text"]
  print("Indices:", indices, "Batch size:", len(text), "Num labels:", len(labels))

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=1024)

  labels_batch = {label: np.zeros(len(text)) for label in labels}
  # examples.keys(): all the column names (keys) in the batch dict "examples"
  # for each k in labels, a new key-value pair k: examples[k] is added to labels_batch
  labels_batch.update({k: examples[k] for k in examples.keys() if k in labels})

  #print(f"labels_batch: {type(labels_batch)} {len(labels_batch)} {labels_batch}")
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    #print("idx=", idx, label)
    labels_matrix[:, idx] = labels_batch[label] # sets the idx-th column of labels_matrix with the values from labels_batch[label]

  print("First row of labels_matrix:", labels_matrix[0])

  # Add labels to encoding
  encoding["labels"] = labels_matrix.tolist()

  return encoding

def preprocess_data_ONE(example):
  # take a batch of texts
  text = example["text"]

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=1024)
  #print(encoding)

  # add labels
  labels_batch = {k: example[k] for k in example.keys() if k in labels} # examples.keys(): all the column names (keys) in the batch dict "examples"
                                                                            # for each k in labels, a new key-value pair k: examples[k] is added to labels_batch
  print(labels_batch)
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  #print(labels_matrix.shape)
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  #encoding["labels"] = labels_matrix.tolist()

  return encoding

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [110]:
#dataset.map(preprocess_data_ONE, batched=False)

In [111]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names, with_indices=True)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127] Batch size: 128 Num labels: 42
First row of labels_matrix: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] Batch size: 18 Num labels: 42
First row of labels_matrix: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53] Batch size: 54 Num labels: 42
First row of labels_matrix: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [112]:
print(f"encoded_dataset['train']: {type(encoded_dataset['train'])} {encoded_dataset['train'].shape}")
idx_list_14=[]
idx_list_16=[]
for i, row in enumerate(encoded_dataset['train']['labels']):
  if (row[14] == 1.0 and sum(row)<2.0):
    idx_list_14.append(i)
    print(f"Row {i}: {row}")
  if (row[16] == 1.0 and sum(row)<2.0):
    idx_list_16.append(i)
    print(f"Row {i}: {row}")
print(f"idx_list_14: {idx_list_14}")
print(f"idx_list_16: {idx_list_16}")

example = encoded_dataset['train'][0]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
example = encoded_dataset['train'][44]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
example = encoded_dataset['train'][65]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
example = encoded_dataset['train'][66]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
example = encoded_dataset['train'][93]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
example = encoded_dataset['train'][100]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
print()
example = encoded_dataset['train'][47]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
example = encoded_dataset['train'][90]
print(tokenizer.decode(example['input_ids']))
print(example['labels'])


encoded_dataset['train']: <class 'datasets.arrow_dataset.Dataset'> (128, 4)
Row 0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Row 44: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Row 47: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Row 65: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Row 66: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [113]:
example = encoded_dataset['train'][0]
print(f"example.keys(): {example.keys()}")
print(f"example['input_ids']: {example['input_ids']}")
print(f"example['token_type_ids']: {example['token_type_ids']}")
print(f"example['attention_mask']: {example['attention_mask']}")
print(f"example['labels']: {example['labels']}")

example.keys(): dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
example['input_ids']: [101, 1999, 3388, 2819, 1011, 2613, 3527, 13728, 2368, 1011, 24296, 6112, 3992, 24296, 6112, 1999, 3388, 2819, 1011, 2613, 3527, 13728, 2368, 2115, 3853, 2000, 2490, 1996, 27258, 3930, 1997, 2256, 24296, 3218, 1010, 2057, 2024, 2559, 2005, 2195, 24296, 6112, 6145, 1012, 2182, 1005, 1055, 2129, 2017, 1005, 2222, 2191, 4254, 1024, 2005, 3469, 3934, 2017, 2147, 1999, 2136, 2007, 2060, 24296, 6112, 6145, 1010, 6112, 5576, 8160, 1998, 2622, 10489, 2000, 4339, 1037, 2047, 3112, 2466, 1012, 2057, 2064, 4175, 2006, 2017, 2005, 1996, 2658, 7375, 1997, 1996, 8518, 18011, 2000, 2017, 1012, 2005, 3760, 6112, 3934, 2017, 2024, 1999, 3715, 2005, 1996, 2440, 8147, 1024, 2017, 18012, 2115, 7396, 2013, 1996, 2640, 2127, 2010, 5576, 2003, 3929, 6515, 1012, 1999, 2804, 2000, 24853, 1010, 2017, 2036, 3749, 2353, 1011, 2240, 2490, 2000, 6304, 1998, 8628, 1012, 19309, 2003, 1999, 2115, 9165, 1012, 20

In [114]:
tokenizer.decode(example['input_ids'])

"[CLS] inetum - realdolmen - azure cloud engineer azure cloud inetum - realdolmen your function to support the exponential growth of our azure practice, we are looking for several azure cloud engineers. here's how you'll make impact : for larger projects you work in team with other azure cloud engineers, cloud solution architects and project managers to write a new success story. we can count on you for the professional implementation of the tasks entrusted to you. for smaller cloud projects you are in charge for the full engagement : you advise your client from the design until his solution is fully operational. in addition to consultancy, you also offer third - line support to customers and colleagues. automation is in your genes. you always strive to make your life and that of the operation engineers easier by automating everything. infrastructure - as - code does not sound weird to you. you test thoroughly, and you don't compromise on quality. you adhere to realdolmens best practic

In [115]:
example = encoded_dataset['train'][0]
print(example["labels"])


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [116]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]


['157']

In [117]:
raise Exception("STOP")

Exception: STOP

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [ ]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

In [ ]:
encoded_dataset['train']['input_ids'][0]

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

Let's start training!

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

## Inference

Let's test the model on a new sentence:

In [ ]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)